# Import libraries

In [1]:
%load_ext autoreload

In [2]:
import pandas as pd
from utils_mmp import get_df_mmp, get_delta_potency, get_similarity, get_unique_mmps
%autoreload 2

# Load compound data

In [3]:
chembl_high_conf_tids = pd.read_csv("./../dataset/chembl_33_pIC50.csv").rename(columns={'nonstereo_aromatic_smiles': 'smiles'})
display(chembl_high_conf_tids)

,smiles,standard_type,pPot,cid,tid
0,Brc1cc2c(NCc3ccccc3)ncnc2s1,IC50,6.617983,CHEMBL3416599,203
1,Brc1cc2c(NCc3ccccn3)ncnc2s1,IC50,5.102153,CHEMBL3416616,203
2,Brc1cc2c(NCc3cccs3)ncnc2s1,IC50,5.862013,CHEMBL3416619,203
3,Brc1cc2c(NCc3ccncc3)ncnc2s1,IC50,5.410833,CHEMBL3416614,203
4,Brc1cc2c(Nc3ccccc3)ncnc2s1,IC50,7.096910,CHEMBL3416621,203
...,...,...,...,...,...
16524,c1csc(-c2n[nH]c3c2Cc2ccccc2-3)c1,IC50,6.031517,CHEMBL212899,279
16525,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.575118,CHEMBL3582232,220
16526,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.490797,CHEMBL3582223,220
16527,c1ncc(-c2cc3c(cn2)[nH]c2ncc(-c4ccc(CN5CCCCC5)c...,IC50,6.304518,CHEMBL3582224,220


# Load MMP data and generate MMP datasets for each target

In [ ]:
df_mmp_final = pd.DataFrame()
for tid in chembl_high_conf_tids.tid.unique()[:10]:

    df_mms_tid = pd.read_csv(f"./mmp_results/top10/df_mmp_{tid}.csv")
    #display(df_mms_tid)

    for trial in range(10):
        mms_final = get_unique_mmps(df_mms_tid, random_state=trial)
        mmp_unique_final = get_df_mmp(mms_final, random_state=trial).reset_index()
        mmp_unique_final['tid'] = tid
        mmp_unique_final['mmp_id'] = [x for x in range(len(mmp_unique_final))]
        mmp_unique_final['dpot'] = mmp_unique_final.apply(lambda x: get_delta_potency(chembl_high_conf_tids, x.cid_1, x.cid_2, x.tid), axis=1)
        mmp_unique_final['similarity'] = mmp_unique_final.apply(lambda x: get_similarity(chembl_high_conf_tids, x.cid_1, x.cid_2, x.tid), axis=1)
        mmp_unique_final['mmp_trial'] = trial
        df_mmp_final = pd.concat([df_mmp_final, mmp_unique_final])
        
df_mmp_final.to_csv("./df_mmp_folder/df_mmp_final.csv", index=False)

# Check if there are any MMP compound duplicates

In [8]:
for tid in df_mmp_final.tid.unique():
    df_tid = df_mmp_final.query(f'tid == {tid}')
    for t in range(10):
        df_tid_t = df_tid.query(f'mmp_trial == {t}')
        for i in df_tid_t.cid_1.tolist():
            if i in df_tid_t.cid_2.tolist():
                print(i)
        for x in df_tid_t.cid_2.tolist():
            if x in df_tid_t.cid_1.tolist():
                print(x)